In [22]:

from pairwise_probit import PairwiseGP
import torch
from variational_gp import BinaryClassificationGP
from aepsych.config import Config
from gpytorch.likelihoods import BernoulliLikelihood
from sklearn.datasets import make_classification


X, y = make_classification(
            n_samples=10,
            n_features=1,
            n_redundant=0,
            n_informative=1,
            random_state=1,
            n_clusters_per_class=1,
        )
X, y = torch.Tensor(X), torch.Tensor(y)

       


class PairwiseGPModel(PairwiseGP):
    pass


pairwise_model = PairwiseGPModel(X, y)

vars(pairwise_model)

IndexError: tuple index out of range

In [ ]:
binary_X, binary_y = make_classification(
            n_samples=10,
            n_features=1,
            n_redundant=0,
            n_informative=1,
            random_state=1,
            n_clusters_per_class=1,
        )
binary_X, binary_y = torch.Tensor(binary_X), torch.Tensor(binary_y).reshape(-1, 1)

       
binary_model = BinaryClassificationGP(
    train_X=binary_X, train_Y=binary_y, likelihood=BernoulliLikelihood(), inducing_points=10
)

vars(binary_model)

{'_num_outputs': 1,
 '_input_batch_shape': torch.Size([]),
 '_aug_batch_shape': torch.Size([]),
 '_is_custom_likelihood': True,
 '_inducing_point_allocator': <botorch.models.utils.inducing_point_allocators.GreedyVarianceReduction at 0x174675d6ee0>,
 'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': OrderedDict([('model',
               _SingleTaskVariationalGP(
                 (variational_strategy): VariationalStrategy(
                   (_variational_distribution): CholeskyVariationalDistribution()
                 )
                 (mean_module): ConstantMean()
                 (covar_module): ScaleKernel(
                   (base_kernel